In [1]:
import sys
print(sys.version)
print("executable: ")
print(sys.executable)
import tensorflow as tf
print(tf.__version__)
import tensorflow_datasets as tfds
print(tfds.__version__)
#print("TFDS version:", tfds.__version__)
from tensorflow.keras import layers, models
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
import matplotlib.pyplot as plt
#import tensorflow.keras as keras

3.8.10 (default, Nov 22 2023, 10:22:35) 
[GCC 9.4.0]
executable: 
/usr/bin/python


2025-01-31 23:24:39.348145: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-31 23:24:39.462547: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-01-31 23:24:40.426599: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/lib:/usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-01-31 23:24:40.426668

2.11.0
4.9.2


In [3]:
mnist_train, mnist_test = tfds.load('mnist', split=['train', 'test'], as_supervised=True)
def preprocess_image(image, label):
    #image = image
    image = tf.cast(image, tf.float32) 
    label = tf.one_hot(label, depth=10)  # One-hot encode the label
    return image, label
mnist_train = mnist_train.map(preprocess_image) #now train is shared among atn and cnn
mnist_test = mnist_test.map(preprocess_image)
mnist_train_atn = mnist_train.batch(32).prefetch(tf.data.AUTOTUNE)
mnist_test_atn = mnist_test.batch(32).prefetch(tf.data.AUTOTUNE) #not used now since test is done in other code still

In [4]:
#defining the network
input_width = 28
input_height = 28
input_channels = 1
input_pixels = 784

n_conv1 = 32
n_conv2 = 64
stride_conv1 = 1
stride_conv2 = 1
conv1_k = 5
conv2_k = 5
max_pool1_k = 2
max_pool2_k = 2

n_hidden = 1024
n_out = 10

input_size_to_hidden = (input_width//(max_pool1_k*max_pool2_k)) * (input_height//(max_pool1_k*max_pool2_k)) *n_conv2

In [5]:
#initialising the weights with random values
# NOT USED IN THIS CODE WITH KERAS IMPLEMENTATION
weights = {
    "wc1" : tf.Variable(tf.random.normal([conv1_k, conv1_k, input_channels, n_conv1])),
    "wc2" : tf.Variable(tf.random.normal([conv2_k, conv2_k, n_conv1, n_conv2])),
    "wh1" : tf.Variable(tf.random.normal([input_size_to_hidden, n_hidden])),
    "wo" : tf.Variable(tf.random.normal([n_hidden, n_out]))
}

biases = {
    "bc1" : tf.Variable(tf.random.normal([n_conv1])),
    "bc2" : tf.Variable(tf.random.normal([n_conv2])),
    "bh1" : tf.Variable(tf.random.normal([n_hidden])),
    "bo" : tf.Variable(tf.random.normal([n_out])),
}

In [6]:
#functions for layers
def conv(x, weights, bias, strides = 1):
    out = tf.nn.conv2d(x, weights, padding="SAME", strides = [1, strides, strides, 1])
    out = tf.nn.bias_add(out, bias)
    out = tf.nn.relu(out)
    return out

def maxpooling(x, k = 2):
    return tf.nn.max_pool(x, padding = "SAME", ksize = [1, k, k, 1], strides = [1, k, k, 1])

In [7]:
#function for forward prop
#This function is edited now with keras functionality. weights are now handled internally but can still be accessed with get_weights method
def cnn(x):
    # Define the layers using Keras API
    #x = layers.Reshape((input_height, input_width, input_channels))(x)
    
    conv1 = layers.Conv2D(n_conv1, kernel_size=(conv1_k, conv1_k), strides=stride_conv1, padding="same", activation="relu")(x)
    conv1_pool = layers.MaxPooling2D(pool_size=(max_pool1_k, max_pool1_k))(conv1)

    conv2 = layers.Conv2D(n_conv2, kernel_size=(conv2_k, conv2_k), strides=stride_conv2, padding="same", activation="relu")(conv1_pool)
    conv2_pool = layers.MaxPooling2D(pool_size=(max_pool2_k, max_pool2_k))(conv2)

    # Flatten the output from convolutional layers
    flatten = layers.Flatten()(conv2_pool)

    # Fully connected layer
    hidden = layers.Dense(n_hidden, activation="relu")(flatten)

    # Dropout
    dropout = layers.Dropout(0.2)(hidden)

    # Output layer (logits)
    output = layers.Dense(n_out, activation = 'softmax')(dropout)
    
    return output


In [8]:
#x = tf.keras.Input(shape=(input_pixels,))
x = tf.keras.Input(shape=(input_height, input_width, input_channels))
y = tf.keras.Input(shape=(n_out,))
#pred is the model
pred = cnn(x)

In [9]:
model = models.Model(inputs=[x], outputs=pred)
optimizer_cnn = tf.keras.optimizers.Adam(learning_rate=0.001) #changed from 0.01 to 0.001
model.compile(optimizer=optimizer_cnn, loss='categorical_crossentropy', metrics=['accuracy'])
train_data = mnist_train #it will be given in batches by the create_partly_adv_data method
initial_train_data = mnist_train.batch(32)
test_data = mnist_test.batch(32)


In [10]:
# @ layers with 2 deconvolution layers
def build_atn(input_shape):
    model = tf.keras.Sequential([
        # First convolutional layers to capture features
        layers.Conv2D(64, (3, 3), activation='relu', padding='same', input_shape=input_shape),
        layers.MaxPooling2D((2, 2)),  # Downsample (14, 14, 64)

        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),  # Downsample (7, 7, 128)

        # Fully connected layer to integrate features
        layers.Flatten(),  # Flatten to shape (7 * 7 * 128 = 6272)
        layers.Dense(6272, activation='relu'),  # Match the reshape target

        # Deconvolution layers to reconstruct adversarial examples
        layers.Reshape((7, 7, 128)),  # Reshape for deconvolution
        layers.Conv2DTranspose(128, (3, 3), activation='relu', strides=2, padding='same'),  # (14, 14, 128)
        layers.Conv2DTranspose(64, (3, 3), activation='relu', strides=2, padding='same'),  # (28, 28, 64)
        
        layers.Conv2DTranspose(1, (3, 3), activation='sigmoid', padding='same'),  # Output layer (28, 28, 1)
        layers.Lambda(lambda x: x * 255)  # Scale to [0, 255]
    ])
    return model

# Build the ATN
input_shape = (28, 28, 1)  # MNIST dataset shape
atn = build_atn(input_shape)

2025-01-31 23:25:09.583863: W tensorflow/tsl/framework/bfc_allocator.cc:479] Allocator (GPU_0_bfc) ran out of memory trying to allocate 150.06MiB (rounded to 157351936)requested by op StatelessRandomUniformV2
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2025-01-31 23:25:09.583910: I tensorflow/tsl/framework/bfc_allocator.cc:1034] BFCAllocator dump for GPU_0_bfc
2025-01-31 23:25:09.583931: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (256): 	Total Chunks: 41, Chunks in use: 41. 10.2KiB allocated for chunks. 10.2KiB in use in bin. 1.3KiB client-requested in use in bin.
2025-01-31 23:25:09.583945: I tensorflow/tsl/framework/bfc_allocator.cc:1041] Bin (512): 	Total Chunks: 1, Chunks in use: 1. 512B allocated for chunks. 512B in use in bin. 512B client-requested in use in bin.
2025-01-31 23:25:09.583959: I tensorflow/tsl/fr

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL _NotOkStatusException REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/core.py(36): __init__
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/gen_stateless_random_ops_v2.py(475): stateless_random_uniform_v2
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/ops/stateless_random_ops.py(501): stateless_random_uniform
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py(1176): op_dispatch_handler
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/traceback_utils.py(150): error_handler
  /usr/local/lib/python3.8/dist-packages/keras/backend.py(2100): random_uniform
  /usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py(662): _generate_init_val
  /usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py(637): __call

In [ ]:
# Define input loss and output loss function
def input_loss(x, x_prime):
    return tf.reduce_mean(tf.square(tf.cast(x, tf.float32) - x_prime))  # Cast x to float32  # L2 loss

def output_loss(y_pred, y_target):
    return tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_target, y_pred)) # crossentropy


In [ ]:
# Combine the models
input_img = tf.keras.Input(shape=input_shape)
x_prime = atn(input_img)
y_pred = model(x_prime)

combined_model = tf.keras.Model(inputs=input_img, outputs=[x_prime, y_pred])

In [ ]:
def get_second_highest_target(y_pred_original, target_class=None):
    top_2_indices = tf.argsort(y_pred_original, direction='DESCENDING', axis=1)[:, :2]
    second_highest_indices = top_2_indices[:, 1]
    # Create one-hot encoding for the second-highest class
    y_target = tf.one_hot(second_highest_indices, depth=y_pred_original.shape[1])
    return y_target

In [ ]:
# Define training procedure
optimizer_atn = tf.keras.optimizers.Adam(learning_rate=0.0001) #hier is ook niet de loss function geinitialiseerd. learning rate verandert naar 0.0001 ipv dynamisch,0.0001 is uit paper
beta = 0.001  # Weight for input-space loss #changed from  #changed from 0.001 to 0.01

@tf.function
def atn_train_step(x, y_original):
    with tf.GradientTape() as tape:
        # Generate adversarial examples
        x_prime = atn(x)
        l_x = input_loss(x, x_prime)
        # Get predictions for original and adversarial inputs
        y_pred_original = model(x, training=False)
        y_pred = model(x_prime, training=False)

        # Compute the second-highest target dynamically
        y_target = get_second_highest_target(y_pred_original)

        # Compute input loss and output loss separately
        
        l_y = output_loss(y_pred, y_target)

        # Compute the combined loss
        loss = beta * l_x + l_y

    # Compute and apply gradients
    gradients = tape.gradient(loss, atn.trainable_variables)
    optimizer_atn.apply_gradients(zip(gradients, atn.trainable_variables))

    # Return total loss, input loss, and output loss
    return loss, l_x, l_y

In [ ]:
from tqdm import tqdm
from time import time

def train_atn():
    
    # Training loop
    epochs = 20 #for experiment reasons, changed from 100 to 3 now
    batch_size = 32
    #beta = 0.1 #this beta is purely for scaling purposes of the output. change beta at atn_train-step
    #optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

    print(f"{'Epoch':<8}{'Total Loss':<12}{'Scaled Input Loss':<12}{'Output Loss':<12}{'Time (s)':<10}")
    print("-" * 50)

    for epoch in range(epochs):
        atn_start_time = time()
        epoch_loss = 0
        epoch_input_loss = 0
        epoch_output_loss = 0
        batch_count = 0

        # Add progress bar for batches
        for x_batch, y_original_batch in tqdm(mnist_train_atn, desc=f"Epoch {epoch + 1}/{epochs}"):
            loss, l_x, l_y = atn_train_step(x_batch, y_original_batch)  # Get all losses
            epoch_loss += loss.numpy()
            epoch_input_loss += l_x.numpy()*0.01 #beta is already applied at atn_train_step
            epoch_output_loss += l_y.numpy()
            batch_count += 1

        # Compute average losses for the epoch
        avg_loss = epoch_loss / batch_count
        avg_input_loss = epoch_input_loss / batch_count
        avg_output_loss = epoch_output_loss / batch_count
        elapsed_time = time() - atn_start_time

        # Print epoch summary
        print(f"{epoch + 1:<8}{avg_loss:<12.4f}{avg_input_loss:<12.4f}{avg_output_loss:<12.4f}{elapsed_time:<10.2f}")
        # Manually apply the ReduceLROnPlateau scheduler
        #if lr_scheduler is not None:
        #    lr_scheduler.on_epoch_end(epoch, logs={'loss': avg_loss})


In [ ]:
def visualize_samples(original, adversarial, index=None):
    """
    Visualize original and adversarial samples side-by-side.
    
    Args:
        original (np.array): The original sample (e.g., shape (28, 28, 1)).
        adversarial (np.array): The adversarial sample (same shape as original).
        index (int, optional): The index of the sample (useful for debugging).
    """
    original = np.squeeze(original)  # Remove extra dimensions if present
    adversarial = np.squeeze(adversarial)  # Remove extra dimensions if present
    
    # Set up the plot
    fig, axs = plt.subplots(1, 2, figsize=(8, 4))
    fig.suptitle(f"Original vs. Adversarial (Index: {index})", fontsize=16)
    
    # Plot original
    axs[0].imshow(original, cmap='gray')
    axs[0].set_title("Original")
    axs[0].axis('off')
    
    # Plot adversarial
    axs[1].imshow(adversarial, cmap='gray')
    axs[1].set_title("Adversarial")
    axs[1].axis('off')
    
    plt.show()

In [ ]:
# Added this cell for adversarial training
#keras.config.enable_unsafe_deserialization()
#atn = tf.keras.models.load_model('jurjen_adversarial_transformation_network_beta-0.01_epochs-50_conv2d_transpose.keras')
#prev method see experiment 6
def create_partly_adv_data(atn, dataset, indices):
    mnist_train_images = np.array([x.numpy() for x, y in dataset]) #as i make a copy, this is safe
    mnist_train_labels = np.array([y.numpy() for x, y in dataset])
    #random should ensure every epoch new images are used to generate adversarial samples
    adversarial_samples = atn.predict(mnist_train_images[indices]) #get list of x prime's
    print(f"How many adversarial samples: {len(adversarial_samples)}")
    mnist_train_images[indices] = adversarial_samples #no condition checking here, its easier (biggest difference with experiment 6
    #recreate the tf.data.Dataset with the modified data
    updated_mnist_train = tf.data.Dataset.from_tensor_slices((mnist_train_images, mnist_train_labels)).batch(32) #return it in batches of 100
    return updated_mnist_train, adversarial_samples, mnist_train_labels[indices]

In [ ]:
def cnn_train_step(batch_data, batch_labels, adversarial_samples, adversarial_labels, batchnumber):
    
    # Train the model on the current batch
    model.train_on_batch(batch_data, batch_labels)


In [ ]:

#first train cnn model briefly so ATN can start training on that checkpoint
model.fit(initial_train_data, epochs=10) #20 naar 5

#train the atn on the checkpoint
model.trainable = False
train_atn()
model.trainable = True

In [11]:
#global adversarial_examples
import sys
#from time import time

count = 0
atn.trainable = False #dit moet je veranderen als hij straks hertraind moet 
fraction = 0.5
num_epochs = 70 #changed from 25 to 3 for experimentation reasons now. #perhaps do multiple epochs if retrain ATN after 2 epochs
for epoch in range(num_epochs):
    cnn_start_time = time()
    print(f"Epoch {epoch + 1}/{num_epochs}")
    #determine indices of adv examples
    num_samples = len(mnist_train)
    num_adversarial = int(fraction * num_samples)
    indices = np.random.choice(num_samples, num_adversarial, replace=False)
    train_data_with_adv, adversarial_examples, adv_true_labels = create_partly_adv_data(atn, train_data, indices) 
    train_data_iterator = iter(train_data_with_adv) # Create an iterator from the dataset
    
    total_loss = 0
    total_accuracy = 0
    
    for batch_number in range(len(train_data_with_adv)): #loop over the number of batches not the data itself as it may change during the loop
        #print(f"batch : {batch_number + 1}")
        try:
            # Get the next batch
            x, labels = next(train_data_iterator)
            loss, accuracy = model.train_on_batch(x, labels)
            
            total_loss += loss
            total_accuracy += accuracy
            # Progress bar display
            progress = int((batch_number + 1) / len(train_data_with_adv) * 30)  # Progress bar length (30 characters)
            sys.stdout.write(
                f"\r[{'=' * progress}{'.' * (30 - progress)}] "
                f"{batch_number + 1}/{len(train_data_with_adv)} - loss: {loss:.4f} - accuracy: {accuracy:.4f}"
            )
            sys.stdout.flush()
            
        except StopIteration:
            break # In case the iterator runs out of data
            
    # End of epoch metrics CNN
    avg_loss = total_loss / len(train_data_with_adv)
    avg_accuracy = total_accuracy / len(train_data_with_adv)
    elapsed_time = time() - cnn_start_time
    print(f"\nEpoch {epoch + 1} completed in {elapsed_time:.2f}s - loss: {avg_loss:.4f} - accuracy: {avg_accuracy:.4f}")
    
    #check per epoch how well the atn can fool the model
    adv_preds_classes = np.argmax(model.predict(adversarial_examples), axis=1)  # Get predicted classes
    adv_true_classes = np.argmax(adv_true_labels, axis=1)  # Get true classes #simply comparing accuracy on how much classes are correct. perhaps using cross entropy loss threshold is better
    # Calculate accuracy on adversarial examples
    adv_accuracy = np.mean(adv_preds_classes == adv_true_classes) * 100 # [True, False, True, True, True], take the mean of that, x100 for percentage
    print(f"Target model accuracy on adversarial samples for epoch {epoch + 1}: {adv_accuracy:.2f}%")
    if adv_accuracy > 95 and count < 20:
        print("Too high accuracy, retrain ATN")
        #atn = build_atn((28, 28, 1)) #reset the atn
        #optimizer_atn = tf.keras.optimizers.Adam(learning_rate=0.0001) #reset optimizer_atn
        atn.trainable = True
        model.trainable = False
        train_atn() #instead of train from scratch, retrain
        atn.trainable = False
        model.trainable = True
        count = count + 1

NameError: name 'atn' is not defined

In [12]:
test_loss, test_acc = model.evaluate(test_data) #dropout is automatically set to 1.0 when calling model.evaluate
print(f'Test accuracy: {test_acc}')
print("summary: ")
model.summary()

2025-01-31 23:25:23.757015: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:433] Could not create cudnn handle: CUDNN_STATUS_NOT_INITIALIZED
2025-01-31 23:25:23.757086: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Possibly insufficient driver version: 555.42.6
2025-01-31 23:25:23.757120: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at conv_ops_fused_impl.h:621 : UNIMPLEMENTED: DNN library is not found.


RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL UnimplementedError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/framework/errors_impl.py(445): __init__
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/execute.py(52): quick_execute
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/polymorphic_function/monomorphic_function.py(378): call
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/polymorphic_function/monomorphic_function.py(1745): _call_flat
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/polymorphic_function/tracing_compiler.py(134): __call__
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py(945): _call
  /usr/local/lib/python3.8/dist-packages/tensorflow/python/eager/polymorphic_function/polymorphic_function.py(880): __call__
  /usr

In [20]:
model.save('Joint_Training_AT.keras')

In [21]:
# Save the ATN with the generated name
atn.save("Joint_Training_ATN.keras")